In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import logging
from IPython.display import HTML, Markdown


logging.basicConfig(level=logging.WARNING) # set your preferred logging level here

# we import the things we need (obviously)
from data.VulnerabilityReport import VulnerabilityReport, create_from_flama_json # this is the function that creates the VulnerabilityReport object from the json data
from ai.LLM.LLMServiceStrategy import LLMServiceStrategy 

from ai.LLM.Strategies.OpenAIService import OpenAIService
from ai.LLM.Strategies.AnthropicService import AnthropicService
from ai.LLM.Strategies.OLLAMAService import OLLAMAService # this is the service that uses ollama to generate solution

model_name=None # but can be anything from here https://ollama.com/library
my_strategy_local = OLLAMAService(model_name=model_name)
my_strategy = AnthropicService()


llm_service_local = LLMServiceStrategy(my_strategy_local)
llm_service = LLMServiceStrategy(my_strategy)

In [ ]:
vulnerabilityReport=VulnerabilityReport.import_from_json('../../../data/VulnerabilityReport_all_llama3_instruct.json')
#Markdown(vulnerabilityReport.sort().get_findings()[4].to_html())

In [ ]:
from ai.Grouping.FindingBatcher import FindingBatcher

findingBatcher = FindingBatcher(llm_service_local)
batches = findingBatcher.create_batches(vulnerabilityReport.get_findings())

In [ ]:
from ai.Grouping.FindingGrouper import FindingGrouper

findingGrouper = FindingGrouper(vulnerabilityReport, llm_service)

grouped_findings = findingGrouper.generate_aggregated_solutions()

In [ ]:
#Markdown(vulnerabilityReport.to_html())
vulnerabilityReport.export_to_json('../../../data/VulnerabilityReport_all_llama3_instruct_with_solution_by_claude.json')